In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.graph_objects as go
import plotly.subplots as sp
import plotly.express as px
import pandas as pd

In [3]:
import geopandas as gpd
path_to_shapefile = './data/ne_10m_admin_0_countries'

# Read the shapefile
world = gpd.read_file(path_to_shapefile)
list_countries = world['NAME']

In [5]:
df = pd.read_csv('./data/World Energy Consumption.csv')

In [6]:
from src.preprocessing.renaming_countries import *
from src.preprocessing.insert_iso_codes import get_iso_code
from src.preprocessing.consumptions_per_capita import retain_consumptions_per_capita
rename_countries(df)
df = drop_countries(df)
get_iso_code(df)
df  = retain_consumptions_per_capita(df)

In [7]:
filtered_list = [s for s in list(df.columns) if 'elec_per_capita' in s.lower()]
strings_to_remove = [ 'other_renewables_elec_per_capita_exc_biofuel']
filtered_list = list(filter(lambda item: item not in strings_to_remove,filtered_list))
formatted_sources = [' '.join(word.capitalize() for word in item.split('_')) for item in filtered_list]

In [10]:
import plotly.express as px
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Assuming 'df' is your DataFrame and it contains a 'country' and 'year' column, along with various metrics.
metrics = filtered_list  # Add or remove metrics as needed. Ensure no None values in this list.

# Define color schemes for different groups
color_schemes = {
    'renewables': px.colors.sequential.Greens,
    'not_renewables': px.colors.sequential.Reds,
    'nuclear': px.colors.sequential.Blues
}

# Group metrics by type for color assignment
metric_groups = {
    'biofuel_elec_per_capita': 'renewables',
    'solar_elec_per_capita': 'renewables',
    'wind_elec_per_capita': 'renewables',
    'other_elec_per_capita': 'renewables',
    'renewables_elec_per_capita': 'renewables',
    'hydro_elec_per_capita': 'renewables',
    'coal_elec_per_capita': 'not_renewables',
    'fossil_elec_per_capita': 'not_renewables',
    'gas_elec_per_capita': 'not_renewables',
    'oil_elec_per_capita': 'not_renewables',
    'low_elec_per_capita': 'not_renewables',
    'nuclear_elec_per_capita': 'nuclear'
}

# Exclude Antarctica from the DataFrame
df = df[df['country'] != 'Antarctica']

# Adjust metrics to fill the grid or manage the overflow as necessary
rows = 3
cols = 4
total_cells = rows * cols

# Create subplots
fig = make_subplots(
    rows=rows, cols=cols,
    subplot_titles=formatted_sources,  # Use metric names as titles, blank for None
    specs=[[{"type": "choropleth"} if metric else None for metric in metrics[row*cols:(row+1)*cols]] for row in range(rows)],  # Only create choropleth where metrics exist
    vertical_spacing=0.01,  # Minimal vertical spacing
    horizontal_spacing=0.01  # Reduced horizontal spacing
)

# List of unique years in the dataset
years = df['year'].unique()[50:]

# Create traces for each year, for each metric that is not None
for year in years:
    filtered_df = df[df['year'] == year]
    
    for i, metric in enumerate(metrics):
        if metric:  # Check if metric is not None
            group = metric_groups.get(metric, 'renewables')  # Default to 'renewables' if not found
            colorscale = color_schemes[group]

            fig.add_trace(
                go.Choropleth(
                    locations=filtered_df['country'],
                    locationmode='country names',
                    z=filtered_df[metric],
                    colorscale=colorscale, 
                    showscale=False,
                    name=str(year)
                ),
                row=i // cols + 1, col=i % cols + 1
            )

# Configure the steps for the slider
steps = []
for i, year in enumerate(years):
    visible = [False] * len(fig.data)
    for j, metric in enumerate(metrics):
        if metric:  
            index = i * total_cells + j  
            if index < len(visible):  
                visible[index] = True

    steps.append({
        "method": "update",
        "args": [{"visible": visible}, {"title": f"World Data Evolution for the Year: {year}"}],
        "label": str(year)
    })

# Horizontal slider configuration
sliders = [{
    'steps': steps,
    'x': 0.1,  # Slider position adjustment if needed
    'len': 0.8,  # Slider length
    'currentvalue': {'prefix': 'Year: '}
}]

fig.update_layout(
    sliders=sliders,
    height=800,  # Reduced height to fit more tightly on a standard screen
    title_text='Global Trends in Per Capita Energy Consumption by Source | 1950 - 2022 | Measured Kw Electricity produced',
)

# Show and save the figure
#fig.show()
fig.write_html("./outputs/Grid_per_capita_consumption.html")